In [2]:
%pip install -q transformers;
%pip install -q torch;

     |████████████████████████████████| 4.7 MB 4.9 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
     |████████████████████████████████| 6.6 MB 45.1 MB/s 


In [3]:
from transformers import BloomTokenizerFast, BloomForCausalLM
from torch.nn import Softmax

from typing import List, Tuple, Iterable, Set, Dict

bloom-(x)b(y) is the version of bloom that have x.y billion parameters
bloom is the full model with 176 billion parameters, and you should only run it on a strong computer with ~500GB of RAM

In [7]:
#@title Select BLOOM's version from the list


version = "bloom-560m" #@param ["bloom", "bloom-7b1", "bloom-6b3", "bloom-3b", "bloom-1b7", "bloom-1b1", "bloom-560m"]
model_name = f"bigscience/{version}"

In [8]:
tokenizer = BloomTokenizerFast.from_pretrained(model_name)
model = BloomForCausalLM.from_pretrained(model_name)

In [9]:
exm_inputs = tokenizer("encode this example", return_tensors="pt")
exm_inputs

{'input_ids': tensor([[47813,  1119,  6635]]), 'attention_mask': tensor([[1, 1, 1]])}

In [10]:
logits_pt_tensor = model(**exm_inputs, labels=exm_inputs["input_ids"]).logits.squeeze(0)
logits_pt_tensor

tensor([[-1.6262, -2.2337,  2.2758,  ..., -1.7645, -1.7644, -1.7643],
        [-2.1960, -2.8377,  4.2420,  ..., -2.3659, -2.3647, -2.3649],
        [-2.0677, -2.5625,  7.4347,  ..., -2.3167, -2.3160, -2.3161]],
       grad_fn=<SqueezeBackward1>)

In [11]:
soft_max_func = Softmax(dim=1)

In [13]:
prob_tensor = soft_max_func(logits_pt_tensor)
prob_tensor

tensor([[6.3068e-08, 3.4355e-08, 3.1220e-06,  ..., 5.4926e-08, 5.4927e-08,
         5.4937e-08],
        [5.1066e-08, 2.6880e-08, 3.1926e-05,  ..., 4.3087e-08, 4.3138e-08,
         4.3129e-08],
        [1.9451e-08, 1.1859e-08, 2.6050e-04,  ..., 1.5165e-08, 1.5175e-08,
         1.5173e-08]], grad_fn=<SoftmaxBackward0>)

In [15]:
import torch
torch.sum(prob_tensor, axis=1)

tensor([1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)

In [16]:
def get_prob_mat(model_name, str_prompt, group_size):
    """Returns the probability matrix as a list of lists of floats"""
    if "bloom" in model_name:
        inputs = tokenizer(str_prompt, return_tensors="pt")
        logits_pt_tensor = model(**inputs, labels=inputs["input_ids"]).logits.squeeze(0)
    prob_tensor = soft_max_func(logits_pt_tensor)
    try:
        prob_tensor = prob_tensor[:group_size, :]
    except IndexError as e:
        raise [e, ValueError("Group size must be smaller than the size of the prompt in tokens")]
    
    prob_mat = [prob_tensor[i, :].tolist() for i in range(group_size)]
    return prob_mat

In [27]:
def combinations(mat):
    """Returns all the lists such that list[j] is in mat[j]
    complexity: prod([len(mat[i]) for i in range(len(mat))])"""
    if len(mat) == 1:
        return [[mat[0][i]] for i in range(len(mat[0]))]
    res = []
    for i in mat[0]:
        for j in combinations(mat[1:]):
            res.append([i] + j)
    return res


def doesnt_have_duplicates(my_list):
    """Return if there isn't a repetition in the list
    complexity: O(n) where n is the length of the list"""
    return len(my_list) == len(set(my_list))


def seq_prob(tokens, prob_mat):
    """Given the probability matrix and a list of tokens
    returns the probability of the sequence
    prob_mat[a][b] is the probability of the token with id b the a-th token in the sequence
    comlexity: O(len(tokens))"""
    probability = 1.0
    sequence_length = len(tokens)
    for i in range(sequence_length):
        curr_token = tokens[i]
        probability *= prob_mat[i][curr_token]
    return probability


def grouped_sampling(prob_mat: List[List[int]], top_p, top_k, group_size) -> Tuple[List[List[int]], List[float]]:
    """given a matrix of probabilities, returns a list of lists of tokens
    the matrixs is of size group_size x vocab_size
    where matrix[i, j] is the probability of token j the i-th token in the group
    samples the tokens such that for each place in the group,
    at most top_k tokens are sampled and at least one token is sampled
    and the added probability of all the tokens is less than or equal top_p
    returns a list of where every item is a tuple of a sequense and probability
    overall complexity of the function in O(group_size * vocab_size * log(vocab_size))"""
    
    # prob_tensor.shape is now (group_size, vocab_size)
    indices = []  # O(1)
    for i in range(group_size): # group_size times
        token_prob = prob_mat[i]  # O(1)
        vocab_size = len(token_prob)  # O(1)
        indexed_prob = list(zip(token_prob, range(vocab_size)))  # O(vocab_size)
        sorted_indexed_prob = sorted(indexed_prob, key=lambda x: x[0], reverse=True)[:top_k] # O(vocab_size*log(vocab_size))
        total_prob: float = sorted_indexed_prob[0][0]  # O(1)
        curr_indices: List[List[int]] = [sorted_indexed_prob[0][1]]
        for prob, token in sorted_indexed_prob:  # top_k times
            if total_prob + prob > top_p:  # O(1)
                break
            total_prob += prob  # O(1)
            curr_indices.append(token)  # O(1)
        indices.append(curr_indices)  # O(1)
    new_sequences: List[List[int]] = combinations(indices)  # theta(prod(len(indices[i]) for i in range(group_size)))
    # len(indices[i]) <= min(top_k, vocab_size)
    # therefore the complexity is O(min(top_k, vocab_size) * group_size)
    filtered_sequences: Iterable[List[int]] = list(filter(doesnt_have_duplicates, new_sequences))
    prob_list: List[Tuple[List[int], float]] = [seq_prob(seq, prob_mat) for seq in filtered_sequences]
    return filtered_sequences, prob_list

In [28]:
from functools import reduce

def flatten(l: list) -> List[int]:
    # Complexity: O(len(the flatten list))
    new_list = []
    for item in l:
        if isinstance(item, int):
            new_list.append(item)
        elif isinstance(item, list):
            new_list.extend(flatten(item))
    return new_list

In [29]:
def remove_duplicates(completions: List[List[int]], probs: List[float]) -> Dict[Tuple[int], float]:
    """Given a list of tokenized answers and the probability of each complition,
    removes every repeated completion and every complition that have repeated tokens"""
    filtered_completions: Dict[Tuple[int], float] = dict()
    for curr_comp, curr_prob in zip(completions, probs):
        if len(curr_comp) == len(set(curr_comp)):
            curr_comp_tuple = tuple(curr_comp)
            filtered_completions[curr_comp_tuple] = curr_prob
    return filtered_completions

In [30]:
def complete(model_name, org_prompt, top_p, top_k, num_groups, group_size, org_prompt_prob = 1.0) -> Dict[Tuple[int], float]:
    """preprocess the prompt and completes the text
    model name: str
    org_prompt: str"""
    if isinstance(org_prompt, str):
        str_prompt = org_prompt
        tokenized_prompt_ten = tokenizer(str_prompt, return_tensors="pt")["input_ids"]
        tokenized_prompt_list = tokenized_prompt_ten.tolist()
    elif isinstance(org_prompt, list):
        tokenized_prompt_list = flatten(org_prompt)
        str_prompt = tokenizer.decode(tokenized_prompt_list)
    else:
        raise ValueError("org_prompt must be a string or list of integers")
    
    prob_mat = get_prob_mat(model_name, str_prompt, group_size)
    tokenized_ans_list, prob_list = grouped_sampling(prob_mat, top_p, top_k, group_size)
    new_prompts: List[List[int]] = [tokenized_prompt_list + ans for ans in tokenized_ans_list]
    if num_groups == 1:
        return remove_duplicates(new_prompts, prob_list)
    all_new_completions: List[List[int]] = []
    all_new_probs: List[float] = []
    for curr_new_prompt, curr_new_prompt_prob in zip(new_prompts, prob_list):
        curr_new_prompt_prob *= org_prompt_prob
        curr_completions = complete(model_name, curr_new_prompt, top_p, top_k, num_groups - 1, group_size, curr_new_prompt_prob)
        for tokens, prob in curr_completions.items():
            all_new_completions.append(tokens)
            all_new_probs.append(prob)
    return remove_duplicates(all_new_completions, all_new_probs)

In [31]:
def token_list_to_str(token_list: List[int], library: str) -> str:
    """decodes a list of tokens ids (integers) to a string"""
    return tokenizer.decode(token_list) 

In [21]:
prompt = 'A large langange model is a ' #@param {type:"string"}
top_p = 0.8 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
top_k = 2 #@param {type:"integer"}
group_size = 4 #@param {type:"integer"}
num_groups = 3 #@param {type:"integer"}

In [32]:
max_num_calls = sum((top_k ** i) for i in range(num_groups))
num_tokens_genrated = num_groups * group_size
pervious_num_calls = sum((top_k ** i) for i in range(num_tokens_genrated))

print(f"The model will generate {num_tokens_genrated} tokens (words or parts of words)")
print(f"It will call the model less than {max_num_calls} calls to the model")
print(f"Previous methods will need {max_num_calls} call to the model to genrate the same text")

The model will generate 12 tokens (words or parts of words)
It will call the model less than 7 calls to the model
Previous methods will need 7 call to the model to genrate the same text


In [33]:
completions = complete(model_name, prompt, top_p, top_k, num_groups, group_size)
completions

In [35]:
completions

{}

In [25]:
max_key = max(completions, key=completions.get)

ValueError: ignored

In [ ]:
tokenizer.decode(max_key)

In [ ]:
sorted_complitions = dict(sorted(completions.items(), key = lambda item: item[1]))
type(sorted_complitions)

In [ ]:
for tokens, prob in sorted_complitions.items():
    print(tokenizer.decode(tokens), prob)